In [58]:
from elasticsearch import Elasticsearch, helpers
import os

# Connect to Elasticsearch
ES_HOST = "http://localhost:9200"
es = Elasticsearch([ES_HOST])

In [59]:
# Delete the index if it exists
if es.indices.exists(index='index_recipes'):
    es.indices.delete(index='index_recipes')

In [60]:
# Define the mapping for the index
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "content": {"type": "text", 
                        "analyzer": "standard",
                        "search_analyzer": "standard"}
        }
    }
}

es.indices.create(index='index_recipes', body=mapping)
print("Index 'index_recipes' with mapping created successfully.")

Index 'index_recipes' with mapping created successfully.


In [ ]:
def bulk_index(directory, index_name):
    actions = []

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            path = os.path.join(directory, filename)
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()

            # Remove file extension for title
            title = os.path.splitext(filename)[0]
            
            action = {
                "_index": index_name,
                "_source": {
                    "title": title,
                    "content": text
                }
            }
            actions.append(action)

    # Bulk index the documents with only one call to Elasticsearch
    if actions:
        helpers.bulk(es, actions)
        print(f"{len(actions)} documents indexed in the '{index_name}' index.")
    else:
        print("No .txt files found in the directory.")

In [62]:
# Index documents from the 'files' directory
bulk_index("files", "index_recipes")

100 documents indexed in the 'index_recipes' index.


In [63]:
def search(query):
   res = es.search(index='index_recipes', body=query)
   print("Search Results:")
   for i, hit in enumerate(res['hits']['hits']):
    res_doc = hit['_source']
    print(f"{i+1}. Title: {res_doc['title']}\n   Content: {res_doc['content']}\n")

In [64]:
search({"query": {"match_all": {}}})

Search Results:
1. Title: Braciole alle erbe con zucca e patate
   Content: Per preparare le braciole alla erbe con zucca e patate, iniziate dal trito di erbe Mescolate gli ingredienti con le dita tagliatele a metà e poi a dadini piccoli Ricavate delle fette di zucca, eliminate la buccia e riducete anch’esse a dadini piccoli Quindi disponete nella pirofila lo strato di dadini di patate e zucca sulle braciole Irrorate la pirofila con un filo d'olio lnfornate la teglia al microonde e cuocete con programma cottura a vapore assistita modalità 3 a 190°. Quindi sfornate le braciole alle erbe con zucca e patate e servite ben calde 

2. Title: Crostino con uova alla benedict e pancetta
   Content: Per preparare il crostino con uova alla benedict e pancetta, iniziate dalla salsa olandese: prendete il burro chiarificato, scioglietelo in un pentolino o nel microonde e tenetelo da parte. Con uno coltellino a lama liscia tritate finemente lo scalogno In un pentolino versate l’aceto l’acqua, il pepe